In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import os
from shapely.wkt import loads
import warnings
warnings.filterwarnings('ignore')

In [2]:
grid_heatmap = pd.read_csv('data/车辆热力分析0712.csv')
Chage_Grid = pd.read_csv('data/订单量0712.csv')
Chage_Grid['总充电量'] = Chage_Grid['总充电量'].str.replace(',','').astype(float)
grid_heatmap['hour'] = grid_heatmap['统计时刻'].str.split(' ').str[1].astype(int)

In [3]:
# 生成栅格范围
def generate_grid(grid_heatmap):
    grid_heatmap['lon'] = grid_heatmap['经纬度编号'].str.split('-').str[0].astype(float)
    grid_heatmap['lat'] = grid_heatmap['经纬度编号'].str.split('-').str[1].astype(float)
    params = {'slon': 120,
    'slat': 30,
    'deltalon': 0.01,
    'deltalat': 0.01,
    'theta': 0,
    'method': 'rect',
    'gridsize': 500}
    grid_heatmap['LONCOL'],grid_heatmap['LATCOL'] = tbd.GPS_to_grid(grid_heatmap['lon'], grid_heatmap['lat'],params)
    grid_heatmap['geometry'] = tbd.grid_to_polygon([grid_heatmap['LONCOL'], grid_heatmap['LATCOL']],params)
    grid_heatmap = gpd.GeoDataFrame(grid_heatmap, geometry='geometry')
    return grid_heatmap
grid_heatmap = generate_grid(grid_heatmap)
Chage_Grid = generate_grid(Chage_Grid)

## 计算订单栅格的面积

In [6]:
grid_shanghai = Chage_Grid[['lon','lat','geometry']].drop_duplicates(subset=['lon','lat'])
grid_shanghai['index'] = [i for i in range(len(grid_shanghai))]
gdf_grid_shanghai = gpd.GeoDataFrame(
    grid_shanghai, 
    geometry='geometry'  # 指定几何列的名称
)
gdf_grid_shanghai.crs = {'init':'epsg:4326'}
tmp = gdf_grid_shanghai.to_crs(epsg=4528)
gdf_grid_shanghai['area'] = tmp.area


## 处理充电站数据

In [4]:
data = gpd.read_file('./data/上海充电桩_中促盟_处理后.geojson')

In [5]:
charge_station = data.copy()
charge_station.rename(columns={'经度':'lon','纬度':'lat'},inplace=True)
charge_station['station_sum'] = charge_station['交流桩数量'] + charge_station['直流桩数量']
charge_station = charge_station[charge_station['充电站类型']!='04环卫物流车专用']

In [10]:
charge_station.crs = {'init':'epsg:4326'}
charge_station_proj = charge_station.to_crs(epsg=4528)
charge_station_proj['geometry'] = charge_station_proj.buffer(500)
charge_station_proj.to_crs(epsg=4326,inplace=True)

In [58]:
charge_station

,myid,充电站名称,充电站位置,充电站投入使用时间,充电站所处道路属性,lon,lat,充电站所属区域分类,充电站类型,交流桩数量,直流桩数量,交流桩总装机功率,直流桩总装机功率,geometry,station_sum
0,0,上海金山石化十五村充电项目,上海市市辖区金山区石化街道金山城市沙滩石化十五村,2023-09-27,01城市内道路,121.347914,30.712635,01充电场站,05其他专用,6.0,0.0,42.0,0,POINT (121.34791 30.71263),6.0
1,1,上海金山石化十一村小区充电站,上海市市辖区金山区石化街道金山城市沙滩石化十一村,2023-09-27,01城市内道路,121.348137,30.715199,01充电场站,05其他专用,6.0,0.0,42.0,0,POINT (121.34814 30.71520),6.0
2,2,石化隆安市场停车场,石化街道沪杭公路7958号,2021-09-16,01城市内道路,121.344560,30.716703,10大型建筑物配建停车场,01公共,12.0,0.0,42.0,0,POINT (121.34456 30.71670),12.0
3,3,上海欣鑫化工充电站,上海市市辖区金山区海金路366号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,2.0,1.0,14.0,60,POINT (121.29606 30.71723),3.0
4,4,上海顶煦化学充电站,上海市市辖区金山区金山区海金路369号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,5.0,1.0,35.0,15,POINT (121.29606 30.71723),6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19611,19611,东方希杰地下车库内部员工新能源车充电站,国定路400号东方希杰地下车库,2023-08-11,None,121.508835,31.299307,01充电场站,01公共,2.0,0.0,14.0,0,POINT (121.50884 31.29931),2.0
19612,19612,加枫路充电站,加枫路26号衡山商务停车场,2022-01-13,None,121.605119,31.313403,03公共机构内部停车场,01公共,0.0,4.0,0.0,240,POINT (121.60512 31.31340),4.0
19613,19613,华润新江湾九里充电站,华润新江湾九里,2023-09-23,None,121.520188,31.338381,07交通枢纽公共停车场,01公共,108.0,0.0,756.0,0,POINT (121.52019 31.33838),108.0
19615,19615,宝钱公路慢充站,宝钱公路,2023-11-27,None,121.119527,31.362807,05写字楼内部停车场,01公共,2.0,0.0,14.0,0,POINT (121.11953 31.36281),2.0


## 栅格和充电站数据的匹配

In [11]:
# 执行空间连接
sjoined = gpd.sjoin(charge_station, gdf_grid_shanghai, how='left', op='within')
sjoined = sjoined.dropna(subset=['index'])
sjoined = sjoined[['index','area','station_sum']]
sjoined = sjoined.groupby(['index','area'])['station_sum'].sum().reset_index()
sjoined['index'] = sjoined['index'].astype(int)

In [12]:
charge_grid = pd.merge(gdf_grid_shanghai,sjoined.drop(columns=['area']),on=['index'],how='left')
charge_grid['station_sum'] = charge_grid['station_sum'].fillna(0)
charge_grid['Charging Station Density'] = charge_grid['area'] / charge_grid['station_sum']
charge_grid

,lon,lat,geometry,index,area,station_sum,Charging Station Density
0,121.46,31.34,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",0,1.055633e+06,585.0,1804.501455
1,121.37,31.32,"POLYGON ((121.36500 31.31500, 121.37500 31.315...",1,1.055793e+06,88.0,11997.652580
2,121.57,31.16,"POLYGON ((121.56500 31.15500, 121.57500 31.155...",2,1.057689e+06,79.0,13388.469899
3,121.39,31.15,"POLYGON ((121.38500 31.14500, 121.39500 31.145...",3,1.057672e+06,82.0,12898.442666
4,121.40,31.29,"POLYGON ((121.39500 31.28500, 121.40500 31.285...",4,1.056143e+06,58.0,18209.361715
...,...,...,...,...,...,...,...
664,121.35,31.29,"POLYGON ((121.34500 31.28500, 121.35500 31.285...",664,1.056111e+06,16.0,66006.907248
665,121.64,31.29,"POLYGON ((121.63500 31.28500, 121.64500 31.285...",665,1.056315e+06,205.0,5152.757341
666,121.36,31.31,"POLYGON ((121.35500 31.30500, 121.36500 31.305...",666,1.055897e+06,71.0,14871.788668
667,121.48,31.13,"POLYGON ((121.47500 31.12500, 121.48500 31.125...",667,1.057952e+06,3.0,352650.831286


## 计算建筑物的面积

In [12]:
## 读取所有的文件
folder_path = './data/上海行政区整合'
geojson_files = []
for filename in os.listdir(folder_path):
    if filename.endswith('.geojson'):
        # 构建完整的文件路径并添加到列表中
        if filename == '行政区划.geojson':
            continue
        file_path = os.path.join(folder_path, filename)
        geojson_files.append(file_path)
gdf_area_distri = gpd.GeoDataFrame()
for file_path in geojson_files:
    tmp = gpd.read_file(file_path)
    gdf_area_distri = pd.concat([gdf_area_distri,tmp],axis=0)
gdf_area_distri

,height,building_id,index_right,name,geometry
0,2,1757,8,嘉定区,"POLYGON ((121.21579 31.46911, 121.21581 31.469..."
1,2,1758,8,嘉定区,"POLYGON ((121.21557 31.46931, 121.21568 31.469..."
2,2,1759,8,嘉定区,"POLYGON ((121.21564 31.46645, 121.21559 31.466..."
3,2,1760,8,嘉定区,"POLYGON ((121.21546 31.46653, 121.21549 31.466..."
4,2,1761,8,嘉定区,"POLYGON ((121.21544 31.46656, 121.21544 31.466..."
...,...,...,...,...,...
9198,50,327918,3,黄浦区,"POLYGON ((121.49959 31.22412, 121.49959 31.224..."
9199,18,327919,3,黄浦区,"POLYGON ((121.50017 31.22288, 121.50013 31.222..."
9200,70,327920,3,黄浦区,"POLYGON ((121.49879 31.22522, 121.49903 31.224..."
9201,160,327921,3,黄浦区,"POLYGON ((121.49855 31.22384, 121.49873 31.223..."


In [165]:
gdf_area_distri.to_csv('./data/shanghai.csv',index=None)

In [168]:
gdf_area_distri.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [27]:
data_area_shanghai = pd.read_csv('./data/shanghai.csv')
data_area_shanghai['geometry'] = data_area_shanghai['geometry'].apply(lambda x: loads(x) if isinstance(x, object) else None)
gdf_area_shanghai = gpd.GeoDataFrame(data_area_shanghai,geometry='geometry')
gdf_area_shanghai

,height,building_id,index_right,name,geometry
0,2,1757,8,嘉定区,"POLYGON ((121.21579 31.46911, 121.21581 31.469..."
1,2,1758,8,嘉定区,"POLYGON ((121.21557 31.46931, 121.21568 31.469..."
2,2,1759,8,嘉定区,"POLYGON ((121.21564 31.46645, 121.21559 31.466..."
3,2,1760,8,嘉定区,"POLYGON ((121.21546 31.46653, 121.21549 31.466..."
4,2,1761,8,嘉定区,"POLYGON ((121.21544 31.46656, 121.21544 31.466..."
...,...,...,...,...,...
1786879,50,327918,3,黄浦区,"POLYGON ((121.49959 31.22412, 121.49959 31.224..."
1786880,18,327919,3,黄浦区,"POLYGON ((121.50017 31.22288, 121.50013 31.222..."
1786881,70,327920,3,黄浦区,"POLYGON ((121.49879 31.22522, 121.49903 31.224..."
1786882,160,327921,3,黄浦区,"POLYGON ((121.49855 31.22384, 121.49873 31.223..."


In [28]:
def calculate_area(gdf):
    gdf.crs = {'init':'epsg:4326'}
    tmp = gdf.to_crs(epsg=4528)
    gdf['area'] = tmp.area
    gdf['area'] = gdf['area'] * gdf['height']
    return gdf 
gdf_area_shanghai = calculate_area(gdf_area_shanghai)
gdf_area_shanghai.drop(columns=['index_right','height'],inplace=True)

In [34]:
charge_station_sjoined = gpd.sjoin(charge_station,gdf_area_shanghai,how='left',op='within')
charge_station_sjoined

,myid,充电站名称,充电站位置,充电站投入使用时间,充电站所处道路属性,lon,lat,充电站所属区域分类,充电站类型,交流桩数量,直流桩数量,交流桩总装机功率,直流桩总装机功率,geometry,station_sum,index_right,building_id,name,area
0,0,上海金山石化十五村充电项目,上海市市辖区金山区石化街道金山城市沙滩石化十五村,2023-09-27,01城市内道路,121.347914,30.712635,01充电场站,05其他专用,6.0,0.0,42.0,0,POINT (121.34791 30.71263),6.0,NaN,NaN,NaN,NaN
1,1,上海金山石化十一村小区充电站,上海市市辖区金山区石化街道金山城市沙滩石化十一村,2023-09-27,01城市内道路,121.348137,30.715199,01充电场站,05其他专用,6.0,0.0,42.0,0,POINT (121.34814 30.71520),6.0,NaN,NaN,NaN,NaN
2,2,石化隆安市场停车场,石化街道沪杭公路7958号,2021-09-16,01城市内道路,121.344560,30.716703,10大型建筑物配建停车场,01公共,12.0,0.0,42.0,0,POINT (121.34456 30.71670),12.0,NaN,NaN,NaN,NaN
3,3,上海欣鑫化工充电站,上海市市辖区金山区海金路366号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,2.0,1.0,14.0,60,POINT (121.29606 30.71723),3.0,1359932.0,246395.0,金山区,4441.157557
4,4,上海顶煦化学充电站,上海市市辖区金山区金山区海金路369号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,5.0,1.0,35.0,15,POINT (121.29606 30.71723),6.0,1359932.0,246395.0,金山区,4441.157557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19611,19611,东方希杰地下车库内部员工新能源车充电站,国定路400号东方希杰地下车库,2023-08-11,None,121.508835,31.299307,01充电场站,01公共,2.0,0.0,14.0,0,POINT (121.50884 31.29931),2.0,NaN,NaN,NaN,NaN
19612,19612,加枫路充电站,加枫路26号衡山商务停车场,2022-01-13,None,121.605119,31.313403,03公共机构内部停车场,01公共,0.0,4.0,0.0,240,POINT (121.60512 31.31340),4.0,NaN,NaN,NaN,NaN
19613,19613,华润新江湾九里充电站,华润新江湾九里,2023-09-23,None,121.520188,31.338381,07交通枢纽公共停车场,01公共,108.0,0.0,756.0,0,POINT (121.52019 31.33838),108.0,NaN,NaN,NaN,NaN
19615,19615,宝钱公路慢充站,宝钱公路,2023-11-27,None,121.119527,31.362807,05写字楼内部停车场,01公共,2.0,0.0,14.0,0,POINT (121.11953 31.36281),2.0,NaN,NaN,NaN,NaN


In [52]:
charge_station_sjoined = charge_station_sjoined[charge_station_sjoined['building_id'].notna()]
charge_station_sjoined

,myid,充电站名称,充电站位置,充电站投入使用时间,充电站所处道路属性,lon,lat,充电站所属区域分类,充电站类型,交流桩数量,直流桩数量,交流桩总装机功率,直流桩总装机功率,geometry,station_sum,index_right,building_id,name,area
3,3,上海欣鑫化工充电站,上海市市辖区金山区海金路366号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,2.0,1.0,14.0,60,POINT (121.29606 30.71723),3.0,1359932.0,246395.0,金山区,4441.157557
4,4,上海顶煦化学充电站,上海市市辖区金山区金山区海金路369号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,5.0,1.0,35.0,15,POINT (121.29606 30.71723),6.0,1359932.0,246395.0,金山区,4441.157557
5,5,上海顶煦化学二期充电站,上海市市辖区金山区海金路369号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,5.0,1.0,35.0,60,POINT (121.29606 30.71723),6.0,1359932.0,246395.0,金山区,4441.157557
10,10,上海经意实业采购充电站,上海市市辖区金山区杭州湾大道,2021-10-09,01城市内道路,121.348129,30.720449,01充电场站,01公共,3.0,0.0,21.0,0,POINT (121.34813 30.72045),3.0,1385118.0,364290.0,金山区,4658.475828
22,22,上海金山区金山卫客运汽车充电站,上海市市辖区金山区山阳镇龙胜东路98号金山卫站,2023-10-31,01城市内道路,121.368576,30.733864,01充电场站,05其他专用,0.0,40.0,0.0,3200,POINT (121.36858 30.73386),40.0,1411602.0,474520.0,金山区,2091.496986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19588,19588,易桩人民路充电站,人民路45号,2022-10-09,None,121.582828,30.925599,03公共机构内部停车场,01公共,6.0,3.0,42.0,180,POINT (121.58283 30.92560),9.0,207167.0,4268.0,奉贤区,729.330342
19593,19593,上海怡通商业广场站,沪南公路668号怡通商业广场B1停车场A区5～10、14、15车位,2020-12-03,01城市内道路,121.563605,31.099719,01充电场站,01公共,0.0,8.0,0.0,320,POINT (121.56360 31.09972),8.0,1234839.0,467606.0,浦东新区,157892.001307
19600,19600,广发大厦停车场充电站,徐汇路555号,2021-11-17,None,121.472712,31.207626,05写字楼内部停车场,01公共,9.0,5.0,63.0,150,POINT (121.47271 31.20763),14.0,1781475.0,249493.0,黄浦区,857.305800
19602,19602,愚园路充电站,长宁区愚园路753号,2023-09-25,None,121.434630,31.220031,03公共机构内部停车场,01公共,0.0,1.0,0.0,60,POINT (121.43463 31.22003),1.0,1762734.0,137663.0,静安区,674.810444


In [57]:
gdf_area_shanghai.crs

<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich